### Implementing Long Short-Term Memory to detect and classify Parkinsons' Freezing of Gait types in time series data

In [74]:
using Pkg

Pkg.add("NNlib")

   Resolving package versions...
  No Changes to `C:\Users\hurub\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\hurub\.julia\environments\v1.8\Manifest.toml`


In [61]:
using Flux
using Flux: @epochs, batch, throttle

using NNlib
using Distributions

In [63]:
mutable struct LongShortTermMemory
    input_size::Int
    outpus_size::Int

    # trainable parameters of the model
    Wf::AbstractMatrix # params of the Forget Gate
    Wi::AbstractMatrix # params of the Input Gate
    Wc::AbstractMatrix # params of the Input Modulation Gate
    Wo::AbstractMatrix # params of the Output Gate

    # biases of the Gates above
    bf::AbstractVector
    bi::AbstractVector
    bc::AbstractVector
    bo::AbstractVector

    # cell state (aka. long-term memory) and hidden state (aka. short-term memory)
    c::AbstractVector
    h::AbstractVector
end

In [67]:
# initialize weights with Gaussian distribution
function LSTM(input_size::Int, hidden_size::Int ; mean=0.0, std=1.0)
    LongShortTermMemory(
        input_size, hidden_size,
        rand(Truncated(Normal(mean, std), 0, 1), (hidden_size, input_size)), # Wf
        rand(Truncated(Normal(mean, std), 0, 1), (hidden_size, input_size)), # Wi
        rand(Truncated(Normal(mean, std), 0, 1), (hidden_size, input_size)), # Wc
        rand(Truncated(Normal(mean, std), 0, 1), (hidden_size, input_size)), # Wo
        rand(Truncated(Normal(mean, std), 0, 1), hidden_size), # bf
        rand(Truncated(Normal(mean, std), 0, 1), hidden_size), # bi
        rand(Truncated(Normal(mean, std), 0, 1), hidden_size), # bc
        rand(Truncated(Normal(mean, std), 0, 1), hidden_size), # bo

        # both the Long-Term and Short-Term memories are initialized with 0 values
        zeros(hidden_size), # c
        zeros(hidden_size)  # h
    )
end

LSTM

In [65]:
# implementing the forwarding method which is used in the Chaining process
function forward(lstm::LongShortTermMemory, x)
    # calculating the Memory modifier values
    f = NNlib.sigmoid_fast(lstm.Wf * x .+ bf) #
    i = NNlib.sigmoid_fast(lstm.Wi * x .+ bi) #
    o = NNlib.sigmoid_fast(lstm.Wo * x .+ bo) #

    # calculating the new memory values
    c = f .* lstm.c .+ i .* NNlib.tanh_fast(lstm.Wc * x .+ bc) # new Long-Term Memory
    h = o .* NNlib.tanh_fast(c) # new Short-Term Memory

    # updating the memory
    lstm.c, lstm.h = c, h

    # returning the hidden parameters for the next layer
    h
end

forward (generic function with 2 methods)

In [69]:
input_size = 10
hidden_size = 20
num_classes = 4 

model = Chain(
    LSTM(input_size, hidden_size),
    Dense(hidden_size, num_classes),
    softmax
)

Chain(
  LongShortTermMemory(10, 20, [0.8187270247953213 0.7437034558082871 … 0.5310576341451857 0.06948511553957118; 0.034388594760129086 0.11011203219259434 … 0.288478060833857 0.20468429534807608; … ; 0.387241047879294 0.7043523564329524 … 0.6646817527348291 0.012376889571117487; 0.44548252714645115 0.4349887693590795 … 0.478296895433531 0.4651957903613904], [0.8887420527669128 0.3226724154627717 … 0.5274955422071969 0.5827336741777697; 0.3421683838843972 0.04633976373134851 … 0.5830183866134266 0.7826051437997072; … ; 0.7825930260759104 0.5396772746779909 … 0.3430953836507762 0.8963115270243956; 0.4636154379426843 0.16701884828094463 … 0.08743218033467748 0.17897025841980793], [0.5173363612812469 0.36256385478911773 … 0.20154640672784077 0.9282548305954437; 0.18164247067614844 0.4720692656705815 … 0.5135633308785238 0.1695685770625405; … ; 0.12370324873578749 0.12475463178568412 … 0.1499922790253042 0.267038611359769; 0.1872758382876212 0.048394715150974446 … 0.35875837645735037 0.

In [72]:
loss(x, y) = Flux.crossentropy(model(x), y)
optimizer = ADAM(0.001)
epochs = 10

;

In [73]:
for epoch in 1:epochs
  for (input, output) in your_data_iterator
    grads = Flux.gradient(params(model)) do
      loss(input, output)
    end
    Flux.update!(optimizer, params(model), grads)
  end
end

LoadError: UndefVarError: your_data_iterator not defined